In [1]:
from numpy.random import rand
import numpy as np
from numpy.random import choice
from numpy import asarray
from numpy import clip
from numpy import argmin
from numpy import min
from numpy import around
from matplotlib import pyplot
import random
from numpy.random import choice
import copy

In [2]:
fragment_size = 10
pop_size = 200
m = 20 # number of fragments
n = 20 # number of sites
l = 20 # number of transactions
RPT = 0.7 #Probability of transaction requested at a site
APF = 0.4 #Probability of fragment accessed by transaction
sol = np.random.randint(0,n,size=(pop_size,m))
sol2 = copy.deepcopy(sol)
site_capacity = m*fragment_size
UCN = 0.7 # random number between 0 and 1
uc = np.random.randint(UCN,UCN*n,size=(n,n))
uc = (uc + uc.T)/2
numberlist = [i for i in range(n)]
weights = [RPT]*20
x = random.choices(numberlist,weights = weights, k = n)
freq = [[0]*l]*n
for i in range(n):
    for j in range(l):
        freq[i][j] = random.randrange(1,1000)
frequency = np.array(freq)    
trfr = [[0]*m]*l
for i in range(l):
    for j in range(m):
        trfr[i][j] = random.randrange(1,fragment_size)
trfr = frequency @ trfr
qfr = np.zeros(shape=(l,m,m))
for i in range(l):
    for j1 in range(m):
        for j2 in range(m):
            qfr[i,j1,j2] = random.randrange(1,fragment_size)
frdep = np.zeros(shape=(m,m))
for i in range(m):
    for j in range(m):
        frdep[i,j] = np.sum(qfr[:,i,j])

In [3]:
def calculation(solution):
    cost1 = 0 # partial cost
    cost2 = 0 # direct cost
    cost = 0
    for j1 in range(m):
        for j2 in range(m):
            cost1 = cost1 + (frdep[j1,j2] + uc[solution[j1],solution[j2]])
    for j in range(m):        
        for q in range(n):   # partialco
            r1 = 0
            for k in range(l):
                r1 = r1 + (frequency[solution[j],k] * trfr[k,j])
                cost2 = cost2 + (uc[solution[j],q] * r1)
    cost = cost1 + cost2
    return cost
    
# def crossover(x,y,cr = 0.8):
#     R = random.randrange(0,1)
#     if R < cr:
#         cutp1 = random.randrange(1,m)
#         cutp2 = random.randrange(1,m)
#         for i in range(cutp1):
#             x[i],y[i] = y[i], x[i]
#         for i in range(cutp2,m):
#             x[i],y[i] = y[i], x[i]
    
#     return x,y

def crossover(x,y,cr = 0.8):
    x1 = copy.deepcopy(x)
    y1= copy.deepcopy(y)
    R = random.randrange(0,1)
    if R < cr:
        cutp1 = random.randrange(1,m)
        cutp2 = random.randrange(1,m)
        for i in range(cutp1):
            x[i],y[i] = y[i], x[i]
        for i in range(cutp2,m):
            x[i],y[i] = y[i], x[i]
    
    if calculation(x) < calculation(x1):
        p = x
    else:
        p = x1
    if calculation(y) < calculation(y1):
        q = y
    else:
        q = y1
    return p,q


def mutation(X,Y):
    if calculation(X) < calculation(Y):
            Z = X
    elif calculation(Y) < calculation(X):
        Z = Y
    else:
        rand = np.random.randint(0,2)
        if rand == 0:
            Z = X
        else:
            Z = Y
    return Z

def VNS(X):
    # neighbour structure will be a matrix of dimension 1Xm. where m = no of fragments = 20
    for k in range(10):
        Y = X
        for p in range(5):
            for q in range(3): # changing 3 sites randomly
                fragment = np.random.randint(0,20)
                site = np.random.randint(0,n)
                Z = Y
                Z[fragment] = site
            if calculation(Z) < calculation(Y):
                Y = Z
        if calculation(Y) < calculation(X):
            X = Y
    return X
                
        
        

In [4]:
COST = [calculation(solution) for solution in sol]
newcost = [(max(COST)-COST[i]) for i in range(pop_size)]
for i in range(1): # number of iteration
    for j in range(200): # for all solution in sol
        candidates = [candidate for candidate in range(pop_size) if candidate != j]
        R1, R2, R3 = sol[choice(candidates, 3, replace=False)]
        A1,B1 = crossover(R2,R3)
        C = mutation(A1,B1)
        A2,B2 = crossover(R1,C)
        C = mutation(A2,B2)
        A3,B3 = crossover(sol[j],C)
        C = mutation(A3,B3)
        D = VNS(C)
        if calculation(D) < calculation(sol[j]):
            sol[j] = D
        
    
        

In [5]:
final_cost = [calculation(i) for i in sol]
print("final solution :", sol[final_cost.index(min(final_cost))] )
print("cost of solution : ",min(final_cost))

final solution : [15 18  9 17  9  9 11  5  5  9  0 17 13  0 18  9  5 12 16  6]
cost of solution :  14474054152469.0


In [6]:
sol2 = np.array(sol2)
initial_cost = [calculation(i) for i in sol2]
print("minimum cost solution in initial population :", sol2[initial_cost.index(min(initial_cost))])
print("cost of solution",min(initial_cost))

minimum cost solution in initial population : [19 11 15 17  5 16  4 16 18 10  0 17 17  0 16 13 11  3 15 12]
cost of solution 14518356830420.0


In [7]:
print("Difference = ",min(initial_cost - min(final_cost)))

Difference =  44302677951.0
